In [21]:
import pandas as pd
import re

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

ratings_original = pd.read_csv('data/ratings_movies.csv', sep=',', index_col=0)
ratings_df = ratings_original.copy()

In [22]:
ratings_df.head(2)

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance


In [23]:
# 8.1
ratings_df['year_release'] = ratings_df['title'].apply(get_year_release)
null = ratings_df['year_release'].shape[0] - ratings_df[ratings_df['year_release'].notnull()]['year_release'].shape[0]
null

18

In [32]:
# 8.2
year_1999 = ratings_df['year_release'] == 1999
ratings_df[year_1999].pivot_table(
    values='rating',
    index='title',
    aggfunc='mean'
    ).sort_values(by='rating').head(3)

,rating
title,
Bloodsport: The Dark Kumite (1999),0.5
Simon Sez (1999),1.0
Chill Factor (1999),1.0


In [35]:
# 8.3
year_2010 = ratings_df['year_release'] == 2010
ratings_df[year_2010].pivot_table(
    values='rating',
    index='genres',
    aggfunc='mean'
    ).sort_values(by='rating').head()

,rating
genres,
Action|Sci-Fi,1.000000
Action|Adventure|Horror,1.500000
Action|Drama|Fantasy,1.500000
Crime|Romance,1.500000
Adventure|Comedy|Fantasy,1.833333


In [38]:
# 8.4
ratings_df.pivot_table(
    values='genres',
    index='userId',
    aggfunc='nunique'
    ).sort_values(by='genres', ascending=False).head(3)

,genres
userId,
599,524
414,482
448,403


In [57]:
# 8.5
ratings_df.groupby(
    by='userId',
)['rating'].agg(['count', 'mean']).sort_values(by=['count', 'mean'], ascending=[True, False]).head(3)

,count,mean
userId,,
53,20,5.0
595,20,4.2
189,20,4.1


In [71]:
# 8.6
year_2018 = ratings_df['year_release'] == 2018
genres_rat = ratings_df[year_2018].pivot_table(
    index='genres',
    values='rating',
    aggfunc=['mean','count']
)
genres_rat[genres_rat['count']['rating'] > 10].sort_values(by=('mean','rating'), ascending=False)

,mean,count
,rating,rating
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [74]:
ratings_df['date'] = pd.to_datetime(ratings_df['date'])
ratings_df['date'].info()

<class 'pandas.core.series.Series'>
Int64Index: 100836 entries, 0 to 100835
Series name: date
Non-Null Count   Dtype         
--------------   -----         
100836 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.5 MB


In [76]:
ratings_df['year_rating'] = ratings_df['date'].dt.year
ratings_df['year_rating'].head(3)

0    2000
1    2000
2    2000
Name: year_rating, dtype: int64

In [83]:
# 8.7
pivot_years_rate = ratings_df.pivot_table(
    values='rating',
    index='genres',
    columns='year_rating'
)
pivot_years_rate.head()

year_rating,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
genres,,,,,,,,,,,,,,,,,,,,,
(no genres listed),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,3.500000,3.000000,3.676471
Action,2.730769,3.538462,NaN,NaN,2.588235,3.0,2.750000,3.833333,2.700000,3.357143,...,NaN,3.5000,4.500000,2.625,2.500000,NaN,3.100000,3.545455,2.637931,2.588235
Action|Adventure,3.454545,4.150000,4.2,4.0,3.738462,3.5,4.304348,3.277778,4.136364,3.413043,...,3.714286,3.4375,3.888889,3.900,3.833333,3.875,3.560976,3.722222,3.527778,3.397436
Action|Adventure|Animation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.000,3.800000,3.750000,3.375000,3.650000
Action|Adventure|Animation|Children,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.000,3.000000,3.500,3.666667,3.833333,3.500000,3.166667


In [81]:
# 8.7
pivot_years_rate.loc['Action|Adventure'].min()

3.2777777777777777

In [88]:
# 8.7
pivot_years_rate.loc['Action|Adventure|Animation|Children|Comedy|IMAX'].sort_values(ascending=False).nlargest(3)

year_rating
2013    5.000000
2018    3.700000
2016    3.666667
Name: Action|Adventure|Animation|Children|Comedy|IMAX, dtype: float64

In [96]:
# 8.7
pivot_years_rate[2018].loc['Animation|Children|Mystery']

5.0

In [97]:
# 8.7
pivot_years_rate.loc['Comedy']

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64